In [2]:
#!pip install surprise
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
import pickle
#sns.set_style("darkgrid")

In [3]:
## import all combined_data files to pandas
df_1 = pd.read_csv('./data/combined_data_1.txt', header = None, names = ['customer_id', 'rating'], usecols = [0,1])
#df2 = pd.read_csv('./data/combined_data_2.txt', header = None, names = ['customer_id', 'rating'], usecols = [0,1])
#df3 = pd.read_csv('./data/combined_data_3.txt', header = None, names = ['customer_id', 'rating'], usecols = [0,1])
#df4 = pd.read_csv('./data/combined_data_4.txt', header = None, names = ['customer_id', 'rating'], usecols = [0,1])
## change rating to float
df_1['rating'] = df_1['rating'].astype(float)
#df2['rating'] = df2['rating'].astype(float)
#df3['rating'] = df3['rating'].astype(float)
#df4['rating'] = df4['rating'].astype(float)
##combine to one large pandas dataframe
df_all = df_1
#data = data.append(df2)
#data = data.append(df3)
#data = data.append(df4)
df_all.index = np.arange(0,len(df_all))

In [4]:
def create_dataframe(use_pickle=True):
    if(use_pickle == True):
        in_pickle = open("pickle/movies_customers_ratings.pickle", "rb")
        data = pd.DataFrame(pickle.load(in_pickle), columns=['movie_id', 'customer_id', 'rating', 'index']).drop(['index'], axis=1)
    else:
        index = 1
        last_movie_id = "1"
        new_data = []
        for customer_id in df_all["customer_id"]:
            # if we find : that means this is a movie_id and not customer_id
            if(customer_id.find(":") > 0):
                movie_id = customer_id.replace(":", "")
                # if(movie_id != last_movie_id):
                #     print(movie_id, " index is ", index)
                last_movie_id = movie_id
            else:
                # we have this row index so use it to get rating
                rating = df_all["rating"][index-1]
                new_data.append([last_movie_id, customer_id, rating, index])
            index += 1
        #output to pickle file
        movies_customers_ratings = open("pickle/movies_customers_ratings.pickle","wb")
        pickle.dump(new_data, movies_customers_ratings)
        movies_customers_ratings.close()
        data = pd.DataFrame(new_data, columns=['movie_id', 'customer_id', 'rating', 'index']).drop(['index'], axis=1)

    # change columns to numerical
    data['movie_id'] = data['movie_id'].astype(int)
    data['customer_id'] = data['customer_id'].astype(int)
    data["rating"] = data["rating"].astype(int)
    return data

data = create_dataframe(use_pickle=True)

In [5]:
data

,movie_id,customer_id,rating
0,1,1488844,3
1,1,822109,5
2,1,885013,4
3,1,30878,4
4,1,823519,3
...,...,...,...
24053759,4499,2591364,2
24053760,4499,1791000,2
24053761,4499,512536,5
24053762,4499,988963,3


In [6]:
print(len(data))

24053764


In [7]:
def all_movie_id_rows(movie_id):
    return data[data['movie_id'] == movie_id]

def all_custumers_id_rows(customer_id):
    return data[data['customer_id'] == customer_id]

In [8]:
all_custumers_id_rows(6)

,movie_id,customer_id,rating
187297,30,6,3
539827,157,6,3
576723,173,6,4
649632,175,6,5
795442,191,6,2
...,...,...,...
23266008,4356,6,4
23526258,4393,6,3
23586501,4406,6,3
23769931,4432,6,3


In [9]:
customers_ratings_stats = data.groupby('customer_id').agg({'rating': ['sum', 'count']}).reset_index()

In [10]:
customers_ratings_stats['avg_rating'] =  customers_ratings_stats['rating']['sum'] / customers_ratings_stats['rating']['count']

In [11]:
print(customers_ratings_stats)

       customer_id rating       avg_rating
                      sum count           
0                6    510   153   3.333333
1                7    793   195   4.066667
2                8     84    21   4.000000
3               10    164    49   3.346939
4               25     14     4   3.500000
...            ...    ...   ...        ...
470753     2649404     49    12   4.083333
470754     2649409     40    10   4.000000
470755     2649421     15     3   5.000000
470756     2649426    301    74   4.067568
470757     2649429    258    62   4.161290

[470758 rows x 4 columns]


# Recommendations with collaborative filtering

In [12]:
reader = Reader()

# get just top 100K rows for faster run time
n_data = Dataset.load_from_df(data[['movie_id', 'customer_id', 'rating']][:], reader)
#data.split(n_folds=3)

svd = SVD()
cross_validate(svd, n_data, measures=['RMSE', 'MAE'])

{'test_rmse': array([0.8817798 , 0.88192341, 0.88248379, 0.88196481, 0.88232605]),
 'test_mae': array([0.68604232, 0.68602275, 0.68641537, 0.68610789, 0.68647331]),
 'fit_time': (930.4507405757904,
  984.6832256317139,
  939.8538687229156,
  942.2527465820312,
  938.3727450370789),
 'test_time': (91.36496424674988,
  103.47529745101929,
  98.98022675514221,
  98.00432777404785,
  105.73217177391052)}

# Average rating stats

In [84]:
def get_avg_rating_less_than(max_rating):
    print(customers_ratings_stats[customers_ratings_stats['avg_rating'] < max_rating])

def get_users_avg_rating(user_id):
    output = customers_ratings_stats[customers_ratings_stats['customer_id'] == user_id]['avg_rating']
    print(output)
    #return output

In [69]:
get_avg_rating_less_than(5)

       customer_id rating       avg_rating
                      sum count           
0                6    510   153   3.333333
1                7    793   195   4.066667
2                8     84    21   4.000000
3               10    164    49   3.346939
4               25     14     4   3.500000
...            ...    ...   ...        ...
470752     2649401    118    30   3.933333
470753     2649404     49    12   4.083333
470754     2649409     40    10   4.000000
470756     2649426    301    74   4.067568
470757     2649429    258    62   4.161290

[459896 rows x 4 columns]


In [70]:
get_avg_rating_less_than(4)

       customer_id rating       avg_rating
                      sum count           
0                6    510   153   3.333333
3               10    164    49   3.346939
4               25     14     4   3.500000
5               33     38    11   3.454545
6               42    142    36   3.944444
...            ...    ...   ...        ...
470747     2649376    133    34   3.911765
470748     2649378    233    72   3.236111
470750     2649384     23     7   3.285714
470751     2649388    257    77   3.337662
470752     2649401    118    30   3.933333

[331845 rows x 4 columns]


In [71]:
get_avg_rating_less_than(3)

       customer_id rating       avg_rating
                      sum count           
40             261     17     7   2.428571
71             392     19     8   2.375000
75             437    211    78   2.705128
117            660      8     3   2.666667
125            692      2     1   2.000000
...            ...    ...   ...        ...
470675     2648907     11     6   1.833333
470691     2649034     82    33   2.484848
470706     2649093    413   139   2.971223
470724     2649240      2     1   2.000000
470737     2649326     12     5   2.400000

[37409 rows x 4 columns]


In [72]:
get_avg_rating_less_than(2)

       customer_id rating       avg_rating
                      sum count           
195           1109     13     7   1.857143
426           2443      2     2   1.000000
540           3174     45    45   1.000000
589           3423    284   145   1.958621
682           3920      1     1   1.000000
...            ...    ...   ...        ...
470101     2645781      2     2   1.000000
470194     2646295      3     2   1.500000
470230     2646481     82    51   1.607843
470553     2648254      2     2   1.000000
470675     2648907     11     6   1.833333

[2686 rows x 4 columns]


In [73]:
get_avg_rating_less_than(1)

Empty DataFrame
Columns: [(customer_id, ), (rating, sum), (rating, count), (avg_rating, )]
Index: []


In [85]:
get_users_avg_rating(1109)
print("")
get_users_avg_rating(2649429)

195    1.857143
Name: avg_rating, dtype: float64

470757    4.16129
Name: avg_rating, dtype: float64
